In [63]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [64]:
tracks = pd.read_csv('spotify_data.csv')

In [65]:
tracks.drop_duplicates(subset=['track_name', 'artist_name'], inplace=True)
tracks.dropna()

,Unnamed: 0,artist_name,track_name,track_id,popularity,year,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0,Jason Mraz,I Won't Give Up,53QF56cjZA9RTuuMZDrSA6,68,2012,acoustic,0.483,0.303,4,-10.058,1,0.0429,0.6940,0.000000,0.1150,0.1390,133.406,240166,3
1,1,Jason Mraz,93 Million Miles,1s8tP3jP4GZcyHDsjvw218,50,2012,acoustic,0.572,0.454,3,-10.286,1,0.0258,0.4770,0.000014,0.0974,0.5150,140.182,216387,4
2,2,Joshua Hyslop,Do Not Let Me Go,7BRCa8MPiyuvr2VU3O9W0F,57,2012,acoustic,0.409,0.234,3,-13.711,1,0.0323,0.3380,0.000050,0.0895,0.1450,139.832,158960,4
3,3,Boyce Avenue,Fast Car,63wsZUhUZLlh1OsyrZq7sz,58,2012,acoustic,0.392,0.251,10,-9.845,1,0.0363,0.8070,0.000000,0.0797,0.5080,204.961,304293,4
4,4,Andrew Belle,Sky's Still Blue,6nXIYClvJAfi6ujLiKqEq8,54,2012,acoustic,0.430,0.791,6,-5.419,0,0.0302,0.0726,0.019300,0.1100,0.2170,171.864,244320,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1159759,1473391,Nicola Conte,Black Spirits,0m27F0IGHLGAWhqd6ccYst,4,2011,trip-hop,0.373,0.742,10,-6.453,0,0.0736,0.3250,0.000141,0.1590,0.5220,107.951,344013,3
1159760,1473392,Nicola Conte,Quiet Dawn,6er9p611eHEcUCU50j7D57,3,2011,trip-hop,0.516,0.675,7,-7.588,0,0.0326,0.7880,0.000129,0.1300,0.2640,119.897,285067,4
1159761,1473393,Amon Tobin,Morning Ms Candis,7jsMMqxy1tt0rH5FzYcZTQ,2,2011,trip-hop,0.491,0.440,5,-8.512,1,0.0274,0.4770,0.003130,0.0936,0.0351,100.076,214253,4
1159762,1473394,Peace Orchestra,Happy Christmas (War Is Over),77lA1InUaXztuRk2vOzD1S,0,2011,trip-hop,0.480,0.405,0,-13.343,1,0.0276,0.4310,0.000063,0.1250,0.2020,133.885,239133,3


In [66]:
tracks['artist_name'] = tracks['artist_name'].str.lower()
tracks['track_name'] = tracks['track_name'].str.lower()

In [67]:
tracks['song_info'] = tracks['track_name'] + ' - ' + tracks['artist_name']
tracks['song_info'] = tracks['song_info'].fillna('')

In [68]:
features = ['danceability', 'energy', 'key', 'loudness', 'mode',
            'speechiness', 'acousticness', 'instrumentalness', 'liveness',
            'valence', 'tempo', 'duration_ms', 'time_signature']

In [70]:
def recommend_songs(input_song_name, input_artist_name, tracks, features, top_n=5):
    input_song_name = input_song_name.lower()
    input_artist_name = input_artist_name.lower()

    input_song_index = tracks[(tracks['track_name'].str.lower() == input_song_name) & 
                              (tracks['artist_name'].str.lower() == input_artist_name)].index

    inputTrack = tracks[(tracks['track_name'].str.lower() == input_song_name) & (tracks['artist_name'].str.lower() == input_artist_name)]
    print("Input track found:")
    print(inputTrack)
    
    if len(input_song_index) == 0:
        print("Song not found.")
        return None
    
    input_song_index = input_song_index[0]

    input_features = tracks.loc[input_song_index, features].values.reshape(1, -1)
    print(input_features)

    cosine_similarities = cosine_similarity(input_features, tracks[features]).flatten()
    print(cosine_similarities)

    top_indices = cosine_similarities.argsort()[-top_n - 1:-1][::-1]

    recommended_songs = tracks.iloc[top_indices]
    return recommended_songs[['track_name', 'artist_name']], cosine_similarities

input_song_name = input("Enter the name of the song: ")
input_artist_name = input("Enter the name of the artist: ")

recommended_songs, cosine_similarities = recommend_songs(input_song_name, input_artist_name, tracks, features)
if recommended_songs is not None and not recommended_songs.empty:
    print("Here are the top recommendations based on the inputted song:")
    print(recommended_songs)
else:
    print("No recommendations found.")

Input track found:
        Unnamed: 0 artist_name track_name                track_id  popularity  \
461992      461992  the weeknd  heartless  6bnF93Rx87YqUBLSgjiMU8          84   

        year genre  danceability  energy  key  ...  mode  speechiness  \
461992  2020   pop         0.537   0.746   10  ...     0         0.15   

        acousticness  instrumentalness  liveness  valence    tempo  \
461992        0.0236          0.000001     0.156    0.252  170.062   

        duration_ms  time_signature               song_info  
461992       198267               4  heartless - the weeknd  

[1 rows x 21 columns]
[[0.537 0.746 10 -5.507 0 0.15 0.0236 1.01e-06 0.156 0.252 170.062 198267
  4]]
[0.99999995 0.99999998 1.         ... 0.99999992 0.99999995 0.9999999 ]
Recommended Songs:
                       track_name       artist_name
327466                  do or die    axel johansson
1019465  do bodda mi (in digital)      roots manuva
1155477         babylon's burning    the valkyrians
3403